# Arrow example

This example illustrates how to convert Arrow data that contains power-grid-model data to NumPy structured arrays, which the power-grid-model requests.

It is by no means intended to provide complete documentation on the topic, but only to show how such conversions could be done.

This example uses `pyarrow.RecordBatch` to demonstrate zero copy operations. The user can choose a `pyarrow.Table` or other structures based on the requirement.

**NOTE:** To run this example, the optional `examples` dependencies are required:

```sh
pip install .[examples]
```

In [1]:
%%capture cap --no-stderr
from IPython.display import display

from power_grid_model import (
    PowerGridModel,
    initialize_array,
    CalculationMethod,
    power_grid_meta_data,
    ComponentType,
    DatasetType,
)
import pyarrow as pa
import pandas as pd
import numpy as np

ZERO_COPY_ERROR_MSG = "Zero-copy conversion requested, but the data types do not match."


## Model

For clarity, a simple network is created. More complex cases work similarly and can be found in the other examples:

```
node 1 ---- line 4 ---- node 2 ----line 5 ---- node 3
   |                       |                      |
source 6               sym_load 7             sym_load 8
```

## Single symmetric calculations

Construct the input data for the model and construct the actual model.

Arrow uses a columnar data format while the power-grid-model offers both: row based or columnar data format.
Converting to/from columnar data can enable having zero copies to be produced while atleast one copy would be produced for row-based data conversions.

### List the power-grid-model data types

See which attributes exist for a given component and which data types are used

In [2]:
node_input_dtype = initialize_array("input", "node", 0).dtype
line_input_dtype = initialize_array("input", "line", 0).dtype
source_input_dtype = initialize_array("input", "source", 0).dtype
asym_load_input_dtype = initialize_array("input", "asym_load", 0).dtype
print("node:", node_input_dtype)
print("line:", line_input_dtype)
print("source:", source_input_dtype)
print("asym_load:", asym_load_input_dtype)

node: {'names': ['id', 'u_rated'], 'formats': ['<i4', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True}
line: {'names': ['id', 'from_node', 'to_node', 'from_status', 'to_status', 'r1', 'x1', 'c1', 'tan1', 'r0', 'x0', 'c0', 'tan0', 'i_n'], 'formats': ['<i4', '<i4', '<i4', 'i1', 'i1', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8', '<f8'], 'offsets': [0, 4, 8, 12, 13, 16, 24, 32, 40, 48, 56, 64, 72, 80], 'itemsize': 88, 'aligned': True}
source: {'names': ['id', 'node', 'status', 'u_ref', 'u_ref_angle', 'sk', 'rx_ratio', 'z01_ratio'], 'formats': ['<i4', '<i4', 'i1', '<f8', '<f8', '<f8', '<f8', '<f8'], 'offsets': [0, 4, 8, 16, 24, 32, 40, 48], 'itemsize': 56, 'aligned': True}
asym_load: {'names': ['id', 'node', 'status', 'type', 'p_specified', 'q_specified'], 'formats': ['<i4', '<i4', 'i1', 'i1', ('<f8', (3,)), ('<f8', (3,))], 'offsets': [0, 4, 8, 9, 16, 40], 'itemsize': 64, 'aligned': True}


The primitive types of each attribute in the arrow tables need to match to make the operation efficient.
We can also create a helper function based on this. 
Note that the asymmetric type of attribute in power-grid-model has a shape of `(3,)` along with a specific type. These represent the 3 phases of electrical system.
Hence asymmetric attributes need to be handled specially.

#### Creating a Schema

Optionally, we can also make this task easier by creating a schema based on the `DatasetType` and `ComponentType` directly from `power_grid_meta_data`. 
They can then directly be used to construct RecordBatches. The user can modify this schema based on the available attributes for each component.

##### Schema with suffixes

Suffixes are added to the asymmetric attribute names to handle them.

In [3]:
def pgm_schema(
    dataset_type: DatasetType, component_type: ComponentType, asym_suffix: tuple[str, str, str] = ("a", "b", "c")
):
    schemas = []
    component_dtype = power_grid_meta_data[dataset_type][component_type].dtype
    for attribute, (dtype, _) in component_dtype.fields.items():
        if dtype.shape == (3,):
            for suffix in asym_suffix:
                schemas.append((f"{attribute}_{suffix}", pa.from_numpy_dtype(dtype.base)))
        else:
            schemas.append((attribute, pa.from_numpy_dtype(dtype)))
    return pa.schema(schemas)


print("-------node scehma-------")
print(pgm_schema(DatasetType.input, ComponentType.node))
print("-------asym load scehma-------")
print(pgm_schema(DatasetType.input, ComponentType.asym_load))

-------node scehma-------
id: int32
u_rated: double
-------asym load scehma-------
id: int32
node: int32
status: int8
type: int8
p_specified_a: double
p_specified_b: double
p_specified_c: double
q_specified_a: double
q_specified_b: double
q_specified_c: double


##### Schema with asymmetric attributes as a struct

The phases can also be combined together in a pyarrow fixed size list.

In [4]:
def pgm_combined_schema(dataset_type: DatasetType, component_type: ComponentType):
    schemas = []
    component_dtype = power_grid_meta_data[dataset_type][component_type].dtype
    for attribute, (dtype, _) in component_dtype.fields.items():
        if dtype.shape == (3,):
            pa_dtype = pa.list_(pa.from_numpy_dtype(dtype.base), 3)
        else:
            pa_dtype = pa.from_numpy_dtype(dtype)
        schemas.append((attribute, pa_dtype))
    return pa.schema(schemas)


print("-------node combined asym scehma-------")
print(pgm_combined_schema(DatasetType.input, ComponentType.node))
print("-------asym load combined asym scehma-------")
print(pgm_combined_schema(DatasetType.input, ComponentType.asym_load))

-------node combined asym scehma-------
id: int32
u_rated: double
-------asym load combined asym scehma-------
id: int32
node: int32
status: int8
type: int8
p_specified: fixed_size_list<item: double>[3]
  child 0, item: double
q_specified: fixed_size_list<item: double>[3]
  child 0, item: double


### Create the grid using Arrow tables

The [power-grid-model documentation on Components](https://power-grid-model.readthedocs.io/en/stable/user_manual/components.html) provides documentation on which components are required and which ones are optional.

Construct the Arrow data as a table with the correct headers and data types.

In [5]:
nodes = pa.record_batch(
    [
        pa.array([1, 2, 3], type=pa.int32()),  # id
        pa.array([10500.0, 10500.0, 10500.0], type=pa.float64()),
    ],
    names=("id", "u_rated"),
)
lines = pa.record_batch(
    [
        pa.array([4, 5], type=pa.int32()),  # id
        pa.array([1, 2], type=pa.int32()),  # from_node
        pa.array([2, 3], type=pa.int32()),  # to_node
        pa.array([1, 1], type=pa.int8()),  # from_status
        pa.array([1, 1], type=pa.int8()),  # to_status
        pa.array([0.11, 0.15], type=pa.float64()),  # r1
        pa.array([0.12, 0.16], type=pa.float64()),  # x1
        pa.array([4.1e-05, 5.4e-05], type=pa.float64()),  # c1
        pa.array([0.1, 0.1], type=pa.float64()),  # tan1
        pa.array([0.01, 0.05], type=pa.float64()),  # r0
        pa.array([0.22, 0.06], type=pa.float64()),  # x0
        pa.array([4.1e-05, 5.4e-05], type=pa.float64()),  # c0
        pa.array([0.4, 0.1], type=pa.float64()),  # tan0
    ],
    names=(
        "id",
        "from_node",
        "to_node",
        "from_status",
        "to_status",
        "r1",
        "x1",
        "c1",
        "tan1",
        "r0",
        "x0",
        "c0",
        "tan0",
    ),
)
sources = pa.record_batch(
    [
        pa.array([6], type=pa.int32()),  # id
        pa.array([1], type=pa.int32()),  # node
        pa.array([1], type=pa.int8()),  # status
        pa.array([1.0], type=pa.float64()),  # u_ref
    ],
    names=("id", "node", "status", "u_ref"),
)
sym_loads = pa.record_batch(
    [
        pa.array([7, 8], type=pa.int32()),  # id
        pa.array([2, 3], type=pa.int32()),  # node
        pa.array([1, 1], type=pa.int8()),  # status
        pa.array([0, 0], type=pa.int8()),  # type
        pa.array([1.0, 2.0], type=pa.float64()),  # p_specified
        pa.array([0.5, 1.5], type=pa.float64()),  # q_specified
    ],
    names=("id", "node", "status", "type", "p_specified", "q_specified"),
)

nodes
# the tables of the other components can be printed similarly

pyarrow.RecordBatch
id: int32
u_rated: double
----
id: [1,2,3]
u_rated: [10500,10500,10500]

### Convert the Arrow data to power-grid-model input data

These Arrow record batch or tables can then be converted to row based or columnar array.

#### Conversion to row based arrays

No direct conversion from Arrow Tables to row based NumPy array exists and a copy is always required. This would not be the most memory efficient approach. 

To ensure support for optional arguments and to prevent version lock, it is recommended to create an empty power-grid-model data set using `initialize_array` and then fill it with the Arrow data.

In [6]:
def arrow_to_numpy_row_based(data: pa.lib.Table, data_type: str, component: str) -> np.ndarray:
    """Convert Arrow data to NumPy data."""
    result = initialize_array(data_type, component, len(data))
    for name, column in zip(data.column_names, data.columns):
        if name in result.dtype.names:
            result[name] = column.to_numpy()
    return result


node_input_row_based = arrow_to_numpy_row_based(nodes, "input", "node")
node_input_row_based

array([(1, 10500.), (2, 10500.), (3, 10500.)],
      dtype={'names': ['id', 'u_rated'], 'formats': ['<i4', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True})

#### Conversion to columnar arrays

For more memory-efficient operations, converting Arrow data to columnar NumPy arrays can be done with zero-copy operations. This ensures that no additional memory is used for the conversion process.

This approach ensures that the data types match and that the conversion is efficient, leveraging the columnar nature of Arrow data. The option of `zero_copy_only` is added in this demo to verify no copies are made. Its usage is not mandatory to ensure zero copy.

In [7]:
def arrow_to_numpy_columnar(
    data: pa.lib.Table, dataset_type: DatasetType, component_type: ComponentType, zero_copy_only: bool = False
) -> np.ndarray:
    """Convert Arrow data to NumPy data."""
    result = {}
    result_dtype = power_grid_meta_data[dataset_type][component_type].dtype
    for name, column in zip(data.column_names, data.columns):
        column_data = column.to_numpy(zero_copy_only=zero_copy_only)
        if zero_copy_only and column_data.dtype != result_dtype[name]:
            raise ValueError(ZERO_COPY_ERROR_MSG)
        result[name] = column_data.astype(result_dtype[name])
    return result


node_input = arrow_to_numpy_columnar(nodes, DatasetType.input, ComponentType.node, zero_copy_only=True)
line_input = arrow_to_numpy_columnar(lines, DatasetType.input, ComponentType.line)
source_input = arrow_to_numpy_columnar(sources, DatasetType.input, ComponentType.source)
sym_load_input = arrow_to_numpy_columnar(sym_loads, DatasetType.input, ComponentType.sym_load)

node_input

{'id': array([1, 2, 3]), 'u_rated': array([10500., 10500., 10500.])}

### Construct the complete input data structure

In [8]:
input_data = {
    "node": node_input,
    "line": line_input,
    "source": source_input,
    "sym_load": sym_load_input,
}

input_data

{'node': {'id': array([1, 2, 3]), 'u_rated': array([10500., 10500., 10500.])},
 'line': {'id': array([4, 5]),
  'from_node': array([1, 2]),
  'to_node': array([2, 3]),
  'from_status': array([1, 1], dtype=int8),
  'to_status': array([1, 1], dtype=int8),
  'r1': array([0.11, 0.15]),
  'x1': array([0.12, 0.16]),
  'c1': array([4.1e-05, 5.4e-05]),
  'tan1': array([0.1, 0.1]),
  'r0': array([0.01, 0.05]),
  'x0': array([0.22, 0.06]),
  'c0': array([4.1e-05, 5.4e-05]),
  'tan0': array([0.4, 0.1])},
 'source': {'id': array([6]),
  'node': array([1]),
  'status': array([1], dtype=int8),
  'u_ref': array([1.])},
 'sym_load': {'id': array([7, 8]),
  'node': array([2, 3]),
  'status': array([1, 1], dtype=int8),
  'type': array([0, 0], dtype=int8),
  'p_specified': array([1., 2.]),
  'q_specified': array([0.5, 1.5])}}

In [9]:
# Optional: validate the input data
from power_grid_model.validation import validate_input_data

validate_input_data(input_data)

### Use the power-grid-model

For more extensive examples, visit the [power-grid-model documentation](https://power-grid-model.readthedocs.io/en/stable/index.html).

In [10]:
# construct the model
model = PowerGridModel(input_data=input_data, system_frequency=50)

# run the calculation
sym_result = model.calculate_power_flow()

# use pandas to tabulate and display the results
pd_sym_node_result = pd.DataFrame(sym_result["node"])
display(pd_sym_node_result)

,id,energized,u_pu,u,u_angle,p,q
0,1,1,1.000325,10503.410670,-0.000067,338777.246279,-3.299419e+06
1,2,1,1.002879,10530.228073,-0.002932,-1.000000,-5.000001e-01
2,3,1,1.004113,10543.184974,-0.004342,-2.000000,-1.500000e+00


### Convert power-grid-model output data to Arrow output data

Using Pandas DataFrames as an intermediate type, constructing Arrow data formats is straightfoward

In [11]:
pa_sym_node_result = pa.record_batch(pd_sym_node_result)

# and similar for other components

pa_sym_node_result

pyarrow.RecordBatch
id: int32
energized: int8
u_pu: double
u: double
u_angle: double
p: double
q: double
----
id: [1,2,3]
energized: [1,1,1]
u_pu: [1.000324825742982,1.0028788641128945,1.004112854674026]
u: [10503.410670301311,10530.228073185392,10543.184974077272]
u_angle: [-0.00006651843181518038,-0.0029317915196012487,-0.004341587216862092]
p: [338777.2462788448,-1.0000002693184182,-1.9999998867105226]
q: [-3299418.661306348,-0.5000000701801947,-1.4999998507078594]

## Single asymmetric calculations

Asymmetric calculations have a tuple of values for some of the attributes and are not easily convertible to pandas data frames.
Instead, one can have a look at the individual components of those attributes and/or flatten the arrays to access all components.

### Asymmetric input

To illustrate the conversion, let's consider a similar grid but with asymmetric loads.

```
node 1 ---- line 4 ---- node 2 ----line 5 ---- node 3
   |                       |                      |
source 6              asym_load 7            asym_load 8
```

In [12]:
asym_load_input_dtype = initialize_array("input", "asym_load", 0).dtype
print("asym_load:", asym_load_input_dtype)

asym_loads = pa.record_batch(
    [
        pa.array([7, 8], type=pa.int32()),  # id
        pa.array([2, 3], type=pa.int32()),  # node
        pa.array([1, 1], type=pa.int8()),  # status
        pa.array([0, 0], type=pa.int8()),  # type
        pa.array([1.0, 2.0], type=pa.float64()),  # p_specified_a
        pa.array([1.0e-2, 2.5], type=pa.float64()),  # p_specified_b
        pa.array([1.1e-2, 4.5e2], type=pa.float64()),  # p_specified_c
        pa.array([0.5, 1.5], type=pa.float64()),  # q_specified_a
        pa.array([1.5e3, 2.5], type=pa.float64()),  # q_specified_b
        pa.array([0.1, 1.5e3], type=pa.float64()),  # q_specified_c
    ],
    names=(
        "id",
        "node",
        "status",
        "type",
        "p_specified_a",
        "p_specified_b",
        "p_specified_c",
        "q_specified_a",
        "q_specified_b",
        "q_specified_c",
    ),
)

asym_loads

asym_load: {'names': ['id', 'node', 'status', 'type', 'p_specified', 'q_specified'], 'formats': ['<i4', '<i4', 'i1', 'i1', ('<f8', (3,)), ('<f8', (3,))], 'offsets': [0, 4, 8, 9, 16, 40], 'itemsize': 64, 'aligned': True}


pyarrow.RecordBatch
id: int32
node: int32
status: int8
type: int8
p_specified_a: double
p_specified_b: double
p_specified_c: double
q_specified_a: double
q_specified_b: double
q_specified_c: double
----
id: [7,8]
node: [2,3]
status: [1,1]
type: [0,0]
p_specified_a: [1,2]
p_specified_b: [0.01,2.5]
p_specified_c: [0.011,450]
q_specified_a: [0.5,1.5]
q_specified_b: [1500,2.5]
q_specified_c: [0.1,1500]

#### Conversion to row based data

The implementation would be similar to [Conversion to row based arrays for symmetric input](#conversion-to-row-based-arrays), with special handling for asymmertic values

In [13]:
def arrow_to_numpy_asym_row_based(
    data: pa.lib.Table,
    dataset_type: DatasetType,
    component_type: ComponentType,
    phases_suffix: tuple[str, str, str] = ("a", "b", "c"),
) -> np.ndarray:
    """Convert asymmetric Arrow data to NumPy data.

    This function is similar to the arrow_to_numpy function, but also supports asymmetric data."""
    result = initialize_array(dataset_type, component_type, len(data))
    for name, (dtype, _) in result.dtype.fields.items():
        if len(dtype.shape) == 0:
            # simple or symmetric data type
            if name in data.column_names:
                result[name] = data.column(name).to_numpy()
        else:
            # asymmetric data type
            for phase_index, phase in enumerate(phases_suffix):
                phase_name = f"{name}_{phase}"

                if phase_name in data.column_names:
                    result[name][:, phase_index] = data.column(phase_name).to_numpy()

    return result


asym_load_input = arrow_to_numpy_asym_row_based(asym_loads, DatasetType.input, ComponentType.asym_load)

asym_load_input

array([(7, 2, 1, 0, [1.0e+00, 1.0e-02, 1.1e-02], [5.0e-01, 1.5e+03, 1.0e-01]),
       (8, 3, 1, 0, [2.0e+00, 2.5e+00, 4.5e+02], [1.5e+00, 2.5e+00, 1.5e+03])],
      dtype={'names': ['id', 'node', 'status', 'type', 'p_specified', 'q_specified'], 'formats': ['<i4', '<i4', 'i1', 'i1', ('<f8', (3,)), ('<f8', (3,))], 'offsets': [0, 4, 8, 9, 16, 40], 'itemsize': 64, 'aligned': True})

#### Conversion to columnar arrays

The implementation would be similar to [Conversion to columnar arrays for symmetric input](#conversion-to-columnar-arrays), with special handling for asymmertic values.
A copy for the 3 phase attributes in this case is always needed.

In [14]:
def arrow_to_numpy_asym_columnar(
    data: pa.lib.Table,
    dataset_type: DatasetType,
    component_type: ComponentType,
    phases_suffix: tuple[str, str, str] = ("a", "b", "c"),
) -> np.ndarray:
    """Convert asymmetric Arrow data to NumPy data.

    This function is similar to the arrow_to_numpy function, but also supports asymmetric data."""
    result = {}
    result_dtype = power_grid_meta_data[dataset_type][component_type].dtype

    for name in result_dtype.names:
        dtype = result_dtype[name]
        if len(dtype.shape) == 0:
            # simple or symmetric data type
            if name in data.column_names:
                column_data = data.column(name).to_numpy()
                result[name] = column_data.astype(result_dtype[name])
        else:
            # asymmetric data type
            for phase_index, phase in enumerate(phases_suffix):
                phase_name = f"{name}_{phase}"
                if phase_name not in data.column_names:
                    continue

                column_data = data.column(phase_name).to_numpy()
                if name not in result:
                    result[name] = np.empty(shape=len(column_data), dtype=result_dtype[name])
                result[name][:, phase_index] = column_data
    return result


asym_load_input_columnar = arrow_to_numpy_asym_columnar(asym_loads, DatasetType.input, ComponentType.asym_load)

asym_load_input_columnar

{'id': array([7, 8]),
 'node': array([2, 3]),
 'status': array([1, 1], dtype=int8),
 'type': array([0, 0], dtype=int8),
 'p_specified': array([[1.0e+00, 1.0e-02, 1.1e-02],
        [2.0e+00, 2.5e+00, 4.5e+02]]),
 'q_specified': array([[5.0e-01, 1.5e+03, 1.0e-01],
        [1.5e+00, 2.5e+00, 1.5e+03]])}

#### Combined representation of 3 phase values

We start from complete 3 phases as a fixed size list array

In [15]:
asym_load_input_dtype = initialize_array("input", "asym_load", 0).dtype
print("asym_load:", asym_load_input_dtype)
# asym_float_type = pa.struct([("a", pa.float64()), ("b", pa.float64()), ("c", pa.float64())])

asym_loads = pa.record_batch(
    [
        pa.array([7, 8], type=pa.int32()),  # id
        pa.array([2, 3], type=pa.int32()),  # node
        pa.array([1, 1], type=pa.int8()),  # status
        pa.array([0, 0], type=pa.int8()),  # type
        pa.array([[1.0, 1.0e-2, 1.1e-2], [2.0, 2.5, 4.5e2]], type=pa.list_(pa.float64(), 3)),  # p_specified
        pa.array([[0.5, 1.5e3, 0.1], [1.5, 2.5, 1.5e3]], type=pa.list_(pa.float64(), 3)),  # q_specified
    ],
    names=("id", "node", "status", "type", "p_specified", "q_specified"),
)

asym_loads

asym_load: {'names': ['id', 'node', 'status', 'type', 'p_specified', 'q_specified'], 'formats': ['<i4', '<i4', 'i1', 'i1', ('<f8', (3,)), ('<f8', (3,))], 'offsets': [0, 4, 8, 9, 16, 40], 'itemsize': 64, 'aligned': True}


pyarrow.RecordBatch
id: int32
node: int32
status: int8
type: int8
p_specified: fixed_size_list<item: double>[3]
  child 0, item: double
q_specified: fixed_size_list<item: double>[3]
  child 0, item: double
----
id: [7,8]
node: [2,3]
status: [1,1]
type: [0,0]
p_specified: [[1,0.01,0.011],[2,2.5,450]]
q_specified: [[0.5,1500,0.1],[1.5,2.5,1500]]

In [16]:
# TODO: Add a function to convert the Arrow data to NumPy data for row based data using fixed size list arrays
# TODO: Added below is a function to convert the Arrow data to NumPy data for columnar data using fixed size list arrays. Should it be kept or removed?

#### Columnar data conversion for asmmetric attribute as list array

In [17]:
def arrow_to_numpy_asym_list_array_columnar(
    data: pa.lib.Table, dataset_type: DatasetType, component_type: ComponentType, zero_copy_only: bool = False
) -> np.ndarray:
    """Convert asymmetric Arrow data to NumPy data.

    This function is similar to the arrow_to_numpy function, but also supports asymmetric data."""
    result = {}
    result_dtype = power_grid_meta_data[dataset_type][component_type].dtype

    for name in result_dtype.names:
        if name not in data.column_names:
            continue
        dtype = result_dtype[name]

        if len(dtype.shape) == 0:
            column_data = data.column(name).to_numpy(zero_copy_only=zero_copy_only)
        else:
            column_data = data.column(name).flatten().to_numpy(zero_copy_only=zero_copy_only).reshape(-1, 3)

        # TODO Find a way to include shape information instead of base dtype
        if zero_copy_only and column_data.dtype.base != dtype.base:
            raise ValueError(ZERO_COPY_ERROR_MSG)
        result[name] = column_data.astype(dtype.base)
    return result


asym_load_input_columnar_asym_list_array = arrow_to_numpy_asym_list_array_columnar(
    asym_loads, DatasetType.input, ComponentType.asym_load, zero_copy_only=True
)

asym_load_input_columnar_asym_list_array

{'id': array([7, 8]),
 'node': array([2, 3]),
 'status': array([1, 1], dtype=int8),
 'type': array([0, 0], dtype=int8),
 'p_specified': array([[1.0e+00, 1.0e-02, 1.1e-02],
        [2.0e+00, 2.5e+00, 4.5e+02]]),
 'q_specified': array([[5.0e-01, 1.5e+03, 1.0e-01],
        [1.5e+00, 2.5e+00, 1.5e+03]])}

### Use the power-grid-model in asymmetric calculations

In [18]:
asym_input_data = {
    "node": node_input,
    "line": line_input,
    "source": source_input,
    "asym_load": asym_load_input,
}

validate_input_data(asym_input_data, symmetric=False)

# construct the model
asym_model = PowerGridModel(input_data=asym_input_data, system_frequency=50)

# run the calculation
asym_result = asym_model.calculate_power_flow(symmetric=False)

# use pandas to display the results, but beware the data types
pd.DataFrame(asym_result["node"]["u_angle"])

,0,1,2
0,-0.000067,-2.094462,2.094328
1,-0.002930,-2.097322,2.091464
2,-0.004338,-2.098733,2.090057


### Convert asymmetric power-grid-model output data to Arrow output data

In [19]:
def numpy_to_arrow(data: np.ndarray) -> pa.lib.table:
    """Convert NumPy data to Arrow data."""
    simple_data_types = []
    multi_value_data_types = []

    for name, (dtype, _) in data.dtype.fields.items():
        if len(dtype.shape) == 0:
            simple_data_types.append(name)
        else:
            multi_value_data_types.append(name)

    result = pa.table(pd.DataFrame(data[simple_data_types]))

    phases = ("a", "b", "c")
    for name in multi_value_data_types:
        column = data[name]

        assert column.shape[1] == len(phases), "Asymmetric data has 3 phase output"

        for phase_index, phase in enumerate(phases):
            sub_column = column[:, phase_index]
            result = result.append_column(f"{name}_{phase}", [pd.Series(sub_column)])

    return result


pa_asym_node_result = numpy_to_arrow(asym_result["node"])

pa_asym_node_result

pyarrow.Table
id: int32
energized: int8
u_pu_a: double
u_pu_b: double
u_pu_c: double
u_a: double
u_b: double
u_c: double
u_angle_a: double
u_angle_b: double
u_angle_c: double
p_a: double
p_b: double
p_c: double
q_a: double
q_b: double
q_c: double
----
id: [[1,2,3]]
energized: [[1,1,1]]
u_pu_a: [[1.0003248257977395,1.0028803762176164,1.0041143008174032]]
u_pu_b: [[1.0003243769486854,1.0028710993140406,1.0041033583077175]]
u_pu_c: [[1.00032436416241,1.0028730789021523,1.0041004935738533]]
u_a: [[6064.146978239599,6079.639179329456,6087.119449677845]]
u_b: [[6064.144257236815,6079.582941090301,6087.053114238262]]
u_c: [[6064.1441797241405,6079.594941705457,6087.035747712152]]
u_angle_a: [[-0.00006651848125694397,-0.0029298831864832267,-0.004337685507209373]]
u_angle_b: [[-2.094461573665813,-2.0973219974462594,-2.098732840554144]]
...

In [20]:
# TODO Add a function to convert the results back to Arrow format for columnar data with individual phases
# TODO Add a function to convert the results back to Arrow format for row data with fixed list array
# TODO Add a function to convert the results back to Arrow format using columnar data with fixed list array

## Batch data

power-grid-model supports batch calculations by providing an `update_data` argument, as shown in [this example](https://power-grid-model.readthedocs.io/en/stable/examples/Power%20Flow%20Example.html#batch-calculation).

Both the `update_data` and the output result are similar to the `input_data` and output data in the above, except that they have another dimension representing the batch index: the first index in the NumPy structured arrays.

This extra index can be represented in Arrow using a [`RecordBatch`](https://arrow.apache.org/docs/cpp/api/table.html#two-dimensional-datasets) or using any other multi-index data format.